# Lab5: Frequent item mining (s437036583)

### Perform Frequent item mining on the following dataset using SON algorithm

https://www.kaggle.com/datasets/irfanasrullah/groceries

## Tasks
 
1. Preprocess the dataset.
1. implement the FP-Growth.
1. implement the PrefixSpan.
1. implement the SON.

In [0]:
import csv

#### Preprocess the dataset

In [0]:
sc = spark.sparkContext
inputFile = 'dbfs:/FileStore/shared_uploads/s437036583@st.uqu.edu.sa/groceries___groceries.csv'

dataset = sc.textFile(inputFile).mapPartitions(lambda x : csv.reader(x))
# Data PreProcessing
header = dataset.first()
# Exclude the header
dataset = dataset.filter(lambda x: x != header)
# Exclude null values
dataset = dataset.map(lambda x: [i for i in x if i])
# Exclude numbers and add index to each transaction
dataset = dataset.map(lambda x: x[1:len(x)]).zipWithIndex()
print(dataset.collect())


[(['citrus fruit', 'semi-finished bread', 'margarine', 'ready soups'], 0), (['tropical fruit', 'yogurt', 'coffee'], 1), (['whole milk'], 2), (['pip fruit', 'yogurt', 'cream cheese', 'meat spreads'], 3), (['other vegetables', 'whole milk', 'condensed milk', 'long life bakery product'], 4), (['whole milk', 'butter', 'yogurt', 'rice', 'abrasive cleaner'], 5), (['rolls/buns'], 6), (['other vegetables', 'UHT-milk', 'rolls/buns', 'bottled beer', 'liquor (appetizer)'], 7), (['potted plants'], 8), (['whole milk', 'cereals'], 9), (['tropical fruit', 'other vegetables', 'white bread', 'bottled water', 'chocolate'], 10), (['citrus fruit', 'tropical fruit', 'whole milk', 'butter', 'curd', 'yogurt', 'flour', 'bottled water', 'dishes'], 11), (['beef'], 12), (['frankfurter', 'rolls/buns', 'soda'], 13), (['chicken', 'tropical fruit'], 14), (['butter', 'sugar', 'fruit/vegetable juice', 'newspapers'], 15), (['fruit/vegetable juice'], 16), (['packaged fruit/vegetables'], 17), (['chocolate'], 18), (['spec

#### Apply FP-Growth Algorithm

In [0]:
from pyspark.ml.fpm import FPGrowth

fpGrowth = FPGrowth(itemsCol="_1", minSupport=0.05, minConfidence=0.2)
model = fpGrowth.fit(dataset.toDF())

# Display frequent itemsets.
model.freqItemsets.show()

# Display generated association rules.
model.associationRules.show()

# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model.transform(dataset.toDF()).show()


+--------------------+----+
|               items|freq|
+--------------------+----+
|     [shopping bags]| 969|
|              [beef]| 516|
|        [rolls/buns]|1809|
|[rolls/buns, whol...| 557|
|       [frankfurter]| 580|
|       [canned beer]| 764|
|            [yogurt]|1372|
|[yogurt, whole milk]| 551|
|            [coffee]| 571|
|[fruit/vegetable ...| 711|
|            [pastry]| 875|
|       [brown bread]| 638|
|      [bottled beer]| 792|
|   [root vegetables]|1072|
|            [butter]| 545|
|        [whole milk]|2513|
|              [pork]| 567|
|[whipped/sour cream]| 705|
|      [citrus fruit]| 814|
|     [bottled water]|1087|
+--------------------+----+
only showing top 20 rows

+------------------+------------------+-------------------+------------------+-------------------+
|        antecedent|        consequent|         confidence|              lift|            support|
+------------------+------------------+-------------------+------------------+-------------------+
|[oth

d ##### As we can see, out of 9835 transactions, I could only obtained 6 Association Rules with minimum 5% Support and minimum 20% Confidence.
##### This means that there isn't enough relations between purchases, hence we cannot rely on this rules to make recommendations for users.

#### Apply PrefixSpan Algorithm

#### Preprocess the dataset

In [0]:
# File location and type
file_location = inputFile
file_type = "text"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

value
"Item(s),Item 1,Item 2,Item 3,Item 4,Item 5,Item 6,Item 7,Item 8,Item 9,Item 10,Item 11,Item 12,Item 13,Item 14,Item 15,Item 16,Item 17,Item 18,Item 19,Item 20,Item 21,Item 22,Item 23,Item 24,Item 25,Item 26,Item 27,Item 28,Item 29,Item 30,Item 31,Item 32"
"4,citrus fruit,semi-finished bread,margarine,ready soups,,,,,,,,,,,,,,,,,,,,,,,,,,,,"
"3,tropical fruit,yogurt,coffee,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"
"1,whole milk,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"
"4,pip fruit,yogurt,cream cheese,meat spreads,,,,,,,,,,,,,,,,,,,,,,,,,,,,"
"4,other vegetables,whole milk,condensed milk,long life bakery product,,,,,,,,,,,,,,,,,,,,,,,,,,,,"
"5,whole milk,butter,yogurt,rice,abrasive cleaner,,,,,,,,,,,,,,,,,,,,,,,,,,,"
"1,rolls/buns,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"
"5,other vegetables,UHT-milk,rolls/buns,bottled beer,liquor (appetizer),,,,,,,,,,,,,,,,,,,,,,,,,,,"
"1,potted plants,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"


d ##### Convert to dataframe and exclude the first row

In [0]:
df = df.collect()
df = df[1:len(df)]

In [0]:
df = spark.sparkContext.parallelize(df)
df = df.toDF()

In [0]:
display(df)

value
"4,citrus fruit,semi-finished bread,margarine,ready soups,,,,,,,,,,,,,,,,,,,,,,,,,,,,"
"3,tropical fruit,yogurt,coffee,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"
"1,whole milk,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"
"4,pip fruit,yogurt,cream cheese,meat spreads,,,,,,,,,,,,,,,,,,,,,,,,,,,,"
"4,other vegetables,whole milk,condensed milk,long life bakery product,,,,,,,,,,,,,,,,,,,,,,,,,,,,"
"5,whole milk,butter,yogurt,rice,abrasive cleaner,,,,,,,,,,,,,,,,,,,,,,,,,,,"
"1,rolls/buns,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"
"5,other vegetables,UHT-milk,rolls/buns,bottled beer,liquor (appetizer),,,,,,,,,,,,,,,,,,,,,,,,,,,"
"1,potted plants,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"
"2,whole milk,cereals,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"


d ##### Split each row into list of string

In [0]:
from pyspark.sql.functions import split, col
df2 = df.select(split(col("value"),",").alias("ValueArray")) \
    .drop("value")
display(df2)

ValueArray
"List(4, citrus fruit, semi-finished bread, margarine, ready soups, , , , , , , , , , , , , , , , , , , , , , , , , , , , )"
"List(3, tropical fruit, yogurt, coffee, , , , , , , , , , , , , , , , , , , , , , , , , , , , , )"
"List(1, whole milk, , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , )"
"List(4, pip fruit, yogurt, cream cheese, meat spreads, , , , , , , , , , , , , , , , , , , , , , , , , , , , )"
"List(4, other vegetables, whole milk, condensed milk, long life bakery product, , , , , , , , , , , , , , , , , , , , , , , , , , , , )"
"List(5, whole milk, butter, yogurt, rice, abrasive cleaner, , , , , , , , , , , , , , , , , , , , , , , , , , , )"
"List(1, rolls/buns, , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , )"
"List(5, other vegetables, UHT-milk, rolls/buns, bottled beer, liquor (appetizer), , , , , , , , , , , , , , , , , , , , , , , , , , , )"
"List(1, potted plants, , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , )"
"List(2, whole milk, cereals, , , , , , , , , , , , , , , , , , , , , , , , , , , , , , )"


d ##### Remove nulls and numbers

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import ArrayType, StructType

def remove_null(lis):
    final_list = []
    lis = [i for i in lis if i]
    lis = lis[1:len(lis)]
    for i in range(len(lis)):
        final_list.append([lis[i]])
    return final_list

remove_nulls = udf(lambda x: remove_null(x), ArrayType(ArrayType(StringType())))
df2 = df2.withColumn('items', remove_nulls(df2.ValueArray))
display(df2)

ValueArray,items
"List(4, citrus fruit, semi-finished bread, margarine, ready soups, , , , , , , , , , , , , , , , , , , , , , , , , , , , )","List(List(citrus fruit), List(semi-finished bread), List(margarine), List(ready soups))"
"List(3, tropical fruit, yogurt, coffee, , , , , , , , , , , , , , , , , , , , , , , , , , , , , )","List(List(tropical fruit), List(yogurt), List(coffee))"
"List(1, whole milk, , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , )",List(List(whole milk))
"List(4, pip fruit, yogurt, cream cheese, meat spreads, , , , , , , , , , , , , , , , , , , , , , , , , , , , )","List(List(pip fruit), List(yogurt), List(cream cheese), List(meat spreads))"
"List(4, other vegetables, whole milk, condensed milk, long life bakery product, , , , , , , , , , , , , , , , , , , , , , , , , , , , )","List(List(other vegetables), List(whole milk), List(condensed milk), List(long life bakery product))"
"List(5, whole milk, butter, yogurt, rice, abrasive cleaner, , , , , , , , , , , , , , , , , , , , , , , , , , , )","List(List(whole milk), List(butter), List(yogurt), List(rice), List(abrasive cleaner))"
"List(1, rolls/buns, , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , )",List(List(rolls/buns))
"List(5, other vegetables, UHT-milk, rolls/buns, bottled beer, liquor (appetizer), , , , , , , , , , , , , , , , , , , , , , , , , , , )","List(List(other vegetables), List(UHT-milk), List(rolls/buns), List(bottled beer), List(liquor (appetizer)))"
"List(1, potted plants, , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , )",List(List(potted plants))
"List(2, whole milk, cereals, , , , , , , , , , , , , , , , , , , , , , , , , , , , , , )","List(List(whole milk), List(cereals))"


In [0]:
df2 = df2.drop('ValueArray')
display(df2)

items
"List(List(citrus fruit), List(semi-finished bread), List(margarine), List(ready soups))"
"List(List(tropical fruit), List(yogurt), List(coffee))"
List(List(whole milk))
"List(List(pip fruit), List(yogurt), List(cream cheese), List(meat spreads))"
"List(List(other vegetables), List(whole milk), List(condensed milk), List(long life bakery product))"
"List(List(whole milk), List(butter), List(yogurt), List(rice), List(abrasive cleaner))"
List(List(rolls/buns))
"List(List(other vegetables), List(UHT-milk), List(rolls/buns), List(bottled beer), List(liquor (appetizer)))"
List(List(potted plants))
"List(List(whole milk), List(cereals))"


In [0]:
from pyspark.ml.fpm import PrefixSpan



prefixSpan = PrefixSpan(sequenceCol='items', minSupport=0.05, maxPatternLength=32,
                        maxLocalProjDBSize=32000000)

# Find frequent sequential patterns.
result = prefixSpan.findFrequentSequentialPatterns(df2)

In [0]:
display(result)

sequence,freq
List(List(sausage)),924
List(List(bottled beer)),792
List(List(pork)),567
List(List(shopping bags)),969
List(List(yogurt)),1372
List(List(frankfurter)),580
List(List(soda)),1715
List(List(napkins)),515
List(List(other vegetables)),1903
List(List(fruit/vegetable juice)),711


#### Sorted in descending order

In [0]:
result = result.sort(col("freq"), ascending=False)
display(result)

sequence,freq
List(List(whole milk)),2513
List(List(other vegetables)),1903
List(List(rolls/buns)),1809
List(List(soda)),1715
List(List(yogurt)),1372
List(List(bottled water)),1087
List(List(root vegetables)),1072
List(List(tropical fruit)),1032
List(List(shopping bags)),969
List(List(sausage)),924


d ##### As we can see, out of 9835 transactions, I could only obtained 31 frequent sequential patterns with minimum 5% Support and maxPatternLength equal to 32.
##### Out of this, there is only three transaction where two differnt items are bought together, and in those transactions, whole milk was on each of them.

d ### a. I would say that the results make sense, given that the data is not big enough. 
### b. There is actually difference regarding the implementation, for example PrefixSpan only accept sequential data type to operate, unlike FP-Growth which accept list data type
### c. I can use these two algorithm to make any types of recommendation systems, for example: in movies and merchants.

#### Apply SON Algorithm